# Stochastic Oscillator

In [ ]:
import sys
import os
import pandas as pd
import numpy as np

In [ ]:
def compute_max_min(df):
    
    df['max_14'] = df['Adj Close']
    df['min_14'] = df['Adj Close']
    
    for i in range(len(df)): 
        if i >= 13:
            df['max_14'].iloc[i] = np.max(df['Adj Close'].iloc[i - 13: i + 1])
            df['min_14'].iloc[i] = np.min(df['Adj Close'].iloc[i - 13: i + 1])
        else:
            df['max_14'].iloc[i] = np.nan
            df['min_14'].iloc[i] = np.nan
    
    return df

def compute_stochastic_oscillator(df):
  
    df["stochastic_oscillator"] = df["Adj Close"]

    for i in range(len(df)):
        if i >= 13:
            df["stochastic_oscillator"].iloc[i] = (df["Adj Close"].iloc[i] - df['min_14'].iloc[i]) / (df["max_14"].iloc[i] - df["min_14"].iloc[i]) * 100
        else:
            df["stochastic_oscillator"].iloc[i] = np.nan

    return df

def identify_signals(df):

    df["buy_signal"] = df["Adj Close"]
    df["sell_signal"] = df["Adj Close"]
  
    for i in range(len(df)):
        if i >= 13:
            df["buy_signal"].iloc[i] = (df["stochastic_oscillator"].iloc[i] <= 20)
            df["sell_signal"].iloc[i] = (df["stochastic_oscillator"].iloc[i] >= 80)
        else:
            df["buy_signal"].iloc[i] = np.nan
            df["sell_signal"].iloc[i] = np.nan

    return df

def calculate_returns(df):
    cost = 0
    revenue = 0
    holding_stock = False
    for i in range(len(df.index)):
        if np.isnan(df["buy_signal"].iloc[i]):
            df["buy_signal"].iloc[i] = False
        if np.isnan(df["sell_signal"].iloc[i]):
            df["sell_signal"].iloc[i] = False
            
    for i in range(len(df.index)):
        if not holding_stock and df["buy_signal"].iloc[i]:
            cost += df["Adj Close"].iloc[i]
            # print("buy at: ", i)
            holding_stock = True

        elif holding_stock and df["sell_signal"].iloc[i]:
            revenue += df["Adj Close"].iloc[i]
            # print("sell at: ", i)
            holding_stock = False

    if holding_stock:
        revenue += df["Adj Close"].iloc[i]
        holding_stock = False

  # store the calculated returns as a variable "returns" and return it
  # recall the formula: returns = revenue/cost - 1
    returns = revenue / cost - 1
    return returns

In [ ]:
def test1(df):
  df = compute_max_min(df)
  fptr.write(df.head(20).to_string())

def test2(df):
  df = compute_max_min(data)
  df = compute_stochastic_oscillator(df)
  fptr.write(df.head(20).to_string())

def test3(df):
  df = compute_max_min(df)
  df = compute_stochastic_oscillator(df)
  df = identify_signals(df)
  fptr.write("Number of Buy Signals: " + str(int(df["buy_signal"].sum())) + "\n")
  fptr.write("Number of Sell Signals: " + str(int(df["sell_signal"].sum())) + "\n")
  fptr.write(df.head(20).to_string())

def test4(df):
  df = compute_max_min(df)
  df = compute_stochastic_oscillator(df)
  df = identify_signals(df)
  returns = calculate_returns(df)
  answer = "The rate of return of this strategy is: {:.4f}".format(returns)
  fptr.write(answer)

if __name__ == '__main__':
    fptr = open(os.environ['OUTPUT_PATH'], 'w')
    tmp = input()
    rows_num = int(input().strip())
    
    data = []
    colnames = list(map(str, input().rstrip().split('\t')))
    for i in range(rows_num):
        line = list(map(str, input().split('\t')))
        line[0] = pd.to_datetime(line[0])
        line[1] = float(line[1])
        data.append(line)    

    data = pd.DataFrame(data, columns = colnames)
    data.set_index('Date')
    
    if tmp == '1':
        test1(data)
    elif tmp == '2':
        test2(data)
    elif tmp == '3':
        test3(data)
    elif tmp == '4':
        test4(data)
    else:
        raise RuntimeError('invalid input')

# Equal Weighted Index Fund Portfolio

In [ ]:
import sys

class State:
    ON = 1
    OFF = 2
    
class OrderState:
    OPEN = 1
    FILLED = 2
    CANCELLED =3

class OrderSide:
    BUY = 1
    SELL = 2
    
class AppBase:
    def __init__(self):
        self.state = State.OFF
    def start(self):
        self.state = State.ON
    def stop(self):
        self.state = State.OFF
    def get_state(self):
        return self.state
    
class MarkerSimulator(AppBase):
    def __init__(self):
        super().__init__()
        self.price = {'ABCD': [51,50,49,52,51],
                      'EFGH': [25,27,26,23,24],
                      'IJKL': [12,11,10, 9, 9],
                      'MNOP': [5,  3, 2, 4, 6]}
        
    def fill_order(self, order):
        order['state']=OrderState.FILLED
        return order
    
    def cancel_order(self,order):
        order['state'] = OrderState.CANCELLED
        return order
    
    def multicast_prices(self):
        return {key:val.pop() for key, val in self.price.items()}


In [ ]:
# init_prices = {
#     'ABCD': 50,
#     'EFGH': 25,
#     'IJKL': 10,
#     'MNOP': 5
# }

# init_positions = {
#     'ABCD': 5,
#     'EFGH': 10,
#     'IJKL': 25,
#     'MNOP': 50
# }

# init_cash = 0


class TradingStrategy(AppBase):
    def __init__(self, name, init_positions, init_prices, init_cash):
        super().__init__()
        self.name = name
        self.counter = 0 # number of orders
        self.orders = []
        
        self.cash = init_cash
        self.positions = init_positions
        self.prices = init_prices
        
        # [YOUR CODE BELOW] initialize the following attribute with the right expression
        self.holdings = {symbol:position*self.prices[symbol] for symbol,position in init_positions.items()}
        self.account_value = self.cash+sum(self.holdings.values())
        
        self.num_stock = len(init_prices)
      
    # [YOUR CODE BELOW] overload a build-in class method, , example: "Trading Strategy: name_of_my_strategy, Account Value: 1000"
    def __repr__(self):
        return f'Trading Strategy: {self.name}, Account Value: {self.account_value}'
    
    
    def update_holdings(self, new_prices):
        """
        @param Dict new_prices: newest stock prices simulated by market simulator
        @return None
        """
        # update the prices attribute 
        self.prices = new_prices
        
        # update holdings dictionary and account_value according to the new market prices
        value = self.cash
        for symbol, position in self.positions.items():
            self.holdings[symbol] = self.positions[symbol]*self.prices[symbol]
            value += self.holdings[symbol]
        self.account_value = value
        
        
    def send_order(self, symbol, price, quantity, side):
        """
        @param Str symbol: stock symbol
        @param Int prices: pre-specified execution price of limit order, manipulated as integer
        @param Int quantity: order quantity
        @param Int side: OrderSide.BUY=1, OrderSide.SELL=2
        @return List: order created
        """
        
        # OrderState.OPEN=1, OrderState.FILLED=2, OrderState.CANCELED=2
        order = {
            'strategyname': self.name,
            'symbol': symbol,
            'side': side,
            'price':price,
            'quantity': quantity,
            'state': OrderState.OPEN 
        }
        
        # add order_id to order, order_id will start from 0
        order['order_id'] = self.counter
        self.counter += 1
        
        # update order attribute
        self.orders.append(order)
        
        # return order
        return order.copy()
        
        
    def rebalance_weight(self):
        """
        @return List[List[]]: a list of orders created for rebalancing the portfolio
        """
        
        orders_sent = []
        
        # calculate the targeted holding amount
        average = self.account_value/self.num_stock
        
        for symbol in self.positions.keys():
            # calculate the targeted position, don't forget int()
            num_share = int(average/self.prices[symbol])
    
            # create the rebalancing orders and append it to order_sent
            if num_share!=self.positions[symbol]:
                order = self.send_order(
                        symbol, self.prices[symbol], abs(num_share-self.positions[symbol]), 
                        OrderSide.BUY if num_share>self.positions[symbol] else OrderSide.SELL
                    )
                orders_sent.append(order.copy())
        
        return orders_sent
    
    
    def handle_order_from_market(self,order_execution, new_prices):
        """
        @param List order_execution: order execution information
        @param Dict new_prices: newest stock prices simulated by market simulator
        @return None
        """
        
        flag = True
        
        for order in self.orders:
            if order['order_id'] == order_execution['order_id']:
                flag = False 
                
                # if the order is filled, update the prices/holdings/cash/value
                # for holdings/values, you can take advantage of update_holdings method above
                if order_execution['state'] == OrderState.FILLED:
                    assert order_execution['price'] == new_prices[order['symbol']]
                    order_side = 3-2*order['side']
                    self.positions[order['symbol']] += order_side*order['quantity']
                    self.cash -= order_side*order['quantity']*order_execution['price']
                    self.update_holdings(new_prices)
                    
                # if the order is canceled, a market order will be executed immediately at new market price 
                # you don't need to take care of the process of executing market order
                # pretend it is executed immediately and just update the prices/holdings/cash/value
                # for holdings/values, you can take advantage of update_holdings method above
                elif order_execution['state'] == OrderState.CANCELLED:
                    order_side = 3-2*order['side']
                    # immediately send a market order and fill it
                    self.positions[order['symbol']] += order_side*order['quantity']
                    self.cash -= order_side*order['quantity']*new_prices[order['symbol']]
                    self.update_holdings(new_prices)
                break
        # don't forget to remove the executed order from you orders attribute
        self.orders.remove(order)
        
        # what if there is no such order? remeber to handle the error
        if flag:
            raise OrderNotFoundException 
        
        
# error handling class
class OrderNotFoundException(Exception):
    pass

In [ ]:
# test case 0
def test_init():
    name = 'Equal Weighted Index Fund Portfolio'
    init_prices = {'ABCD': 50, 'EFGH': 25, 'IJKL': 10, 'MNOP': 5}
    init_positions = {'ABCD': 5, 'EFGH': 10,'IJKL': 25, 'MNOP': 50}
    init_cash = 0
    ts = TradingStrategy(name,init_positions, init_prices, init_cash)
    
    status = "SUCCEED"
    if ts.holdings != {'ABCD': 250, 'EFGH': 250, 'IJKL': 250, 'MNOP': 250}:
        status = "FAIL_holdings, expected: {'ABCD': 250,　'EFGH': 250,　'IJKL': 250,　'MNOP': 250}" 
    if ts.account_value != 1000:
        status = "FAIL_account_value" 
        
    print(status)
    
    
# test case 1 
def test_repr():
    name = 'Equal Weighted Index Fund Portfolio'
    init_prices = {'ABCD': 50, 'EFGH': 25, 'IJKL': 10, 'MNOP': 5}
    init_positions = {'ABCD': 5, 'EFGH': 10,'IJKL': 25, 'MNOP': 50}
    init_cash = 0
    ts = TradingStrategy(name,init_positions, init_prices, init_cash)
    
    if repr(ts) != 'Trading Strategy: Equal Weighted Index Fund Portfolio, Account Value: 1000':
        status = "FAIL_repr, expected: 'Trading Strategy: Equal Weighted Index Fund Portfolio, Account Value: 1000'" 
    else: 
        status = "SUCCEED"

    print(status)
    
# test case 2  
def test_update_holdings():
    name = 'Equal Weighted Index Fund Portfolio'
    init_prices = {'ABCD': 50, 'EFGH': 25, 'IJKL': 10, 'MNOP': 5}
    init_positions = {'ABCD': 5, 'EFGH': 10,'IJKL': 25, 'MNOP': 50}
    init_cash = 0
    ts = TradingStrategy(name,init_positions, init_prices, init_cash)
    sim = MarkerSimulator()
    new_prices = sim.multicast_prices()
    ts.update_holdings(new_prices)

    status = "SUCCEED"
    if ts.holdings != {'ABCD': 255, 'EFGH': 240, 'IJKL': 225, 'MNOP': 300}:
        status = "FAIL_holdings, expected: {'ABCD': 255,　'EFGH': 240,　'IJKL': 225,　'MNOP': 300}" 
    if ts.account_value != 1020:
        status = "FAIL_account_value, expected: 1020" 
    if ts.positions != {'ABCD': 5, 'EFGH': 10, 'IJKL': 25, 'MNOP': 50}:
        status = "FAIL_positions, expected: {'ABCD': 5,　'EFGH': 10,　'IJKL': 25,　'MNOP': 50}" 
    if ts.cash != 0:
        status = "FAIL_cash, expected: 0" 
    if ts.prices != {'ABCD': 51, 'EFGH': 24, 'IJKL': 9, 'MNOP': 6}:
        status = "FAIL_prices, expected {'ABCD': 51,　'EFGH': 24,　'IJKL': 9,　'MNOP': 6}" 
        
    print(status)
    
    
# test case 3     
def test_send_order():
    name = 'Equal Weighted Index Fund Portfolio'
    init_prices = {'ABCD': 50, 'EFGH': 25, 'IJKL': 10, 'MNOP': 5}
    init_positions = {'ABCD': 5, 'EFGH': 10,'IJKL': 25, 'MNOP': 50}
    init_cash = 0
    ts = TradingStrategy(name,init_positions, init_prices, init_cash)
    order = ts.send_order(symbol='ABCD', price=50, quantity=1, side=OrderSide.BUY)
    
    status = "SUCCEED"
    if order != {
        'strategyname': 'Equal Weighted Index Fund Portfolio',
        'symbol': 'ABCD',
        'side': 1,
        'price':50,
        'quantity': 1,
        'state': OrderState.OPEN,
        'order_id': 0
    }:
        status = "FAIL_order, expected:"+"\n"+"{'strategyname': 'Equal Weighted Index Fund Portfolio',　'symbol': 'ABCD',　'side': 1,　'price':50,　'quantity': 1,　'state': 1,　'order_id': 0}"
    if len(ts.orders) != 1 and ts.orders[0] != order:
        status = "FAIL_orders, expected:"+"\n"+"[{'strategyname': 'Equal Weighted Index Fund Portfolio','symbol': 'ABCD',　'side': 1,　'price':50,　'quantity': 1,　'state': 1,　'order_id': 0}]"
    
    print(status)


# test case 4 
def test_rebalance_weight():
    name = 'Equal Weighted Index Fund Portfolio'
    init_prices = {'ABCD': 50, 'EFGH': 25, 'IJKL': 10, 'MNOP': 5}
    init_positions = {'ABCD': 5, 'EFGH': 10,'IJKL': 25, 'MNOP': 50}
    init_cash = 0
    ts = TradingStrategy(name,init_positions, init_prices, init_cash)
    sim = MarkerSimulator()
    new_prices = sim.multicast_prices()
    ts.update_holdings(new_prices)
    orders = ts.rebalance_weight()
    
    status = "SUCCEED"
    if orders != [
        {'strategyname': 'Equal Weighted Index Fund Portfolio', 'symbol': 'IJKL', 'side': 1, 'price': 9, 'quantity': 3, 'state': 1, 'order_id': 0}, 
        {'strategyname': 'Equal Weighted Index Fund Portfolio', 'symbol': 'MNOP', 'side': 2, 'price': 6, 'quantity': 8, 'state': 1, 'order_id': 1}
    ]:
        status = "FAIL_orders, expected:"+"\n"+"[{'strategyname': 'Equal Weighted Index Fund Portfolio',　'symbol': 'IJKL',　'side': 1,　'price': 9,　'quantity': 3,　'state': 1,　'order_id': 0},"+"\n"+"{'strategyname': 'Equal Weighted Index Fund Portfolio',　'symbol': 'MNOP',　'side': 2,　'price': 6,　'quantity': 8,　'state': 1,　'order_id': 1}]"
    if ts.orders != orders:
        status = "FAIL_orders, expected:"+"\n"+"[{'strategyname': 'Equal Weighted Index Fund Portfolio',　'symbol': 'IJKL',　'side': 1,　'price': 9,　'quantity': 3,　'state': 1,　'order_id': 0},"+"\n"+"{'strategyname': 'Equal Weighted Index Fund Portfolio',　'symbol': 'MNOP',　'side': 2,　'price': 6,　'quantity': 8,　'state': 1,　'order_id': 1}]"
    if ts.positions != {'ABCD': 5, 'EFGH': 10, 'IJKL': 25, 'MNOP': 50}:
        status = "FAIL_positions, expected: {'ABCD': 5,　'EFGH': 10,　'IJKL': 25,　'MNOP': 50}"
    if ts.prices != {'ABCD': 51, 'EFGH': 24, 'IJKL': 9, 'MNOP': 6}:
        status = "FAIL_prices, expected: {'ABCD': 51,　'EFGH': 24,　'IJKL': 9,　'MNOP': 6}"
    if ts.holdings != {'ABCD': 255, 'EFGH': 240, 'IJKL': 225, 'MNOP': 300}:
        status = "FAIL_holdings, expected: {'ABCD': 255,　'EFGH': 240,　'IJKL': 225,　'MNOP': 300}" 
    if ts.cash != 0:
        status = "FAIL_cash, expected: 0" 
    if ts.account_value != 1020:
        status = "FAIL_account_value, expected: 1020"   
        
    print(status)
    

# test case 5
def test_handle_order_from_market_filled():
    name = 'Equal Weighted Index Fund Portfolio'
    init_prices = {'ABCD': 50, 'EFGH': 25, 'IJKL': 10, 'MNOP': 5}
    init_positions = {'ABCD': 5, 'EFGH': 10,'IJKL': 25, 'MNOP': 50}
    init_cash = 0
    ts = TradingStrategy(name,init_positions, init_prices, init_cash)
    sim = MarkerSimulator()
    new_prices = sim.multicast_prices()
    ts.update_holdings(new_prices)
    orders = ts.rebalance_weight()
    order_execution = sim.fill_order(orders[0])
    new_prices = sim.multicast_prices()
    ts.handle_order_from_market(order_execution, new_prices)
    
    status = "SUCCEED"
    if ts.orders != [{'strategyname': 'Equal Weighted Index Fund Portfolio', 'symbol': 'MNOP', 'side': 2, 'price': 6, 'quantity': 8, 'state': 1, 'order_id': 1}]:
        status = "FAIL_orders, expected:"+"\n"+"[{'strategyname': 'Equal Weighted Index Fund Portfolio',　'symbol': 'MNOP',　'side': 2,　'price': 6,　'quantity': 8,　'state': 1,　'order_id': 1}]"
    if ts.positions != {'ABCD': 5, 'EFGH': 10, 'IJKL': 28, 'MNOP': 50}:
        status = "FAIL_positions, expected: {'ABCD': 5,　'EFGH': 10,　'IJKL': 28,　'MNOP': 50}"
    if ts.prices != {'ABCD': 52, 'EFGH': 23, 'IJKL': 9, 'MNOP': 4}:
        status = "FAIL_prices, expected: {'ABCD': 52,　'EFGH': 23,　'IJKL': 9,　'MNOP': 4}"
    if ts.holdings != {'ABCD': 260, 'EFGH': 230, 'IJKL': 252, 'MNOP': 200}:
        status = "FAIL_holdings, expected: {'ABCD': 260,　'EFGH': 230,　'IJKL': 252,　'MNOP': 200}" 
    if ts.cash != -27:
        status = "FAIL_cash, expected: -27" 
    if ts.account_value != 915:
        status = "FAIL_account_value, expected: 915" 
    
    print(status)


# test case 6
def test_handle_order_from_market_canceled():
    name = 'Equal Weighted Index Fund Portfolio'
    init_prices = {'ABCD': 50, 'EFGH': 25, 'IJKL': 10, 'MNOP': 5}
    init_positions = {'ABCD': 5, 'EFGH': 10,'IJKL': 25, 'MNOP': 50}
    init_cash = 0
    ts = TradingStrategy(name,init_positions, init_prices, init_cash)
    sim = MarkerSimulator()
    new_prices = sim.multicast_prices()
    ts.update_holdings(new_prices)
    orders = ts.rebalance_weight()
    order_execution = sim.fill_order(orders[0])
    new_prices = sim.multicast_prices()
    ts.handle_order_from_market(order_execution, new_prices)
    order_execution = sim.cancel_order(orders[1])
    new_prices = sim.multicast_prices()
    ts.handle_order_from_market(order_execution, new_prices)
    
    status = "SUCCEED"
    if ts.orders != []:
        status = "FAIL_orders, expected: [　]"
    if ts.positions != {'ABCD': 5, 'EFGH': 10, 'IJKL': 28, 'MNOP': 42}:
        status = "FAIL_positions, expected: {'ABCD': 5,　'EFGH': 10,　'IJKL': 28,　'MNOP': 42}"
    if ts.prices != {'ABCD': 49, 'EFGH': 26, 'IJKL': 10, 'MNOP': 2}:
        status = "FAIL_prices, expected: {'ABCD': 49,　'EFGH': 26,　'IJKL': 10,　'MNOP': 2}"
    if ts.holdings != {'ABCD': 245, 'EFGH': 260, 'IJKL': 280, 'MNOP': 84}:
        status = "FAIL_holdings, expected: {'ABCD': 245,　'EFGH': 260,　'IJKL': 280,　'MNOP': 84}" 
    if ts.cash != -11:
        status = "FAIL_cash, expected: -11" 
    if ts.account_value != 858:
        status = "FAIL_account_value, expected: 858" 
    
    print(status)
    

# test case 7
def test_error_handling():
    name = 'Equal Weighted Index Fund Portfolio'
    init_prices = {'ABCD': 50, 'EFGH': 25, 'IJKL': 10, 'MNOP': 5}
    init_positions = {'ABCD': 5, 'EFGH': 10,'IJKL': 25, 'MNOP': 50}
    init_cash = 0
    ts = TradingStrategy(name,init_positions, init_prices, init_cash)
    order = ts.send_order(symbol='ABCD', price=50, quantity=1, side=OrderSide.BUY)
    sim = MarkerSimulator()
    new_prices = sim.multicast_prices()
    order_execution = sim.fill_order(order)
    try:
        order_execution['order_id'] = 404
        ts.handle_order_from_market(order_execution, new_prices)
        status = "FAIL_error_handling"
        print(status)
    except OrderNotFoundException:
        status = 'SUCCEED'
        print(status)
    except Exception:
        status = "FAIL_error_handling"
        print(status)


if __name__ == '__main__':
    func_name = sys.stdin.readline().strip()
    test_func = globals()[func_name]
    test_func()

# Trading Strategy XXX

In [ ]:
#!/bin/python3

import math
import os
import random
import re
import sys
import pandas as pd
from abc import ABC
from collections import deque
from sklearn.linear_model import LogisticRegression

class DIRECTION:
    BUY=1
    SELL=-1
    HOLD = 0

class base_strategy(ABC):
    def predict(self):
        pass
    def fit(self,price):
        pass



In [ ]:

#All the class strategy you will need to define will inherit from the class base_strategy
# base_strategy inherits from the class ABC, it doesn't matter what it means. It just prevents
# from creating an instance of the class base_strategy without inheriting from this class (abstract class)
#You will need to use the inheritance for each class declaration
#Please DO NOT forget to call the constructor of the base class
#You will need to define at least 3 methods
# The constructor __init__
# The predict function
# The fit function
# The function predict will need to return either
    # DIRECTION.BUY
    # DIRECTION.SELL
    # DIRECTION.HOLD

# In all the strategies, do not forget that you will always need to start by buying
import numpy as np
class strategy1(base_strategy):
    def __init__(self):
        super().__init__()
        self.counter=0
    
    def predict(self, price):
        self.counter += 1
        if self.counter % 8 == 1:
            return DIRECTION.BUY
        elif self.counter % 8 == 5:
            return DIRECTION.SELL
        else:
            return DIRECTION.HOLD


class strategy2(base_strategy):
    def __init__(self):
        super().__init__()
        self.mavg = 0
        self.buy = True
        self.prices = []
    
    def predict(self, price):
        self.prices.append(price['close'])
        if len(self.prices) > 10:
            self.prices.pop(0)
        self.mavg = np.mean(self.prices)
        if(self.mavg>560 and not self.buy):
            self.buy = True
            return DIRECTION.SELL
        if(self.mavg<559 and self.buy):
            self.buy = False
            return DIRECTION.BUY

class strategy3:
    # hint for the predict function
    # if len(your_dataframe) <= 500, you will hold until you receive enough data
    # store the previous price
    # create your input X
    # create your input Y
    # train your logistic regression this way: LogisticRegression(random_state=0).fit(X=X,y=Y)
    # build your X_test with the latest value then use the function predict
    # if prediction == 1 and you were not in the position you can buy
    # if prediction == -1 and not you were in a long position, you can sell
    # if not you can hold
    def __init__(self):
        super().__init__()
        self.counter = 0
        self.prev_price = np.NaN
        self.X = []
        self.Y = []
        self.model = LogisticRegression(random_state=0)
        self.buy = False
        
        
    def fit(self, price):
        self.counter += 1
        self.append_new_row(price)
        if self.counter > 500:
            self.model.fit(self.X[-500:-1], self.Y[-500:-1])
        
    def predict(self, price):
        if self.counter <= 500:
            return DIRECTION.HOLD
        else:
            pred = self.model.predict([self.X[-1]])
        
        if pred > 0 and not self.buy:
            self.buy = True
            return DIRECTION.BUY
        elif pred < 0 and self.buy:
            self.buy = False
            return DIRECTION.SELL
        else:
            return DIRECTION.HOLD
    
    def append_new_row(self, price):
        new_row = [
            1 if price['price'] > self.prev_price else -1,
            price['price'],
            price['open'] - price['close'],
            price['high'] - price['low']
        ]
        self.prev_price = price['price']
        self.X.append(new_row[1:])
        self.Y.append(new_row[0])
        if self.counter > 500:
            self.X.pop(0)
            self.Y.pop(0)
        
        





In [ ]:
class ForLoopBackTester:
    def __init__(self,strat=None):
        self.list_position=[]
        self.list_cash=[]
        self.list_holdings = []
        self.list_total=[]

        self.long_signal=False
        self.position=0
        self.cash=100000
        self.total=0
        self.holdings=0

        self.market_data_count=0
        self.prev_price = None
        self.statistical_model = None
        self.historical_data = pd.DataFrame(columns=['Trade','Price','OpenClose','HighLow'])
        self.strategy = strat



    def onMarketDataReceived(self,price_update):
        if self.strategy:
            self.strategy.fit(price_update)
            predicted_value = self.strategy.predict(price_update)
        else:
            predicted_value = DIRECTION.HOLD

        if predicted_value==DIRECTION.BUY:
            return 'buy'
        if predicted_value==DIRECTION.SELL:
            return 'sell'
        return 'hold'

    def buy_sell_or_hold_something(self,price_update,action):
        if action == 'buy':
            cash_needed = 10 * price_update['price']
            if self.cash - cash_needed >=0:
                print(str(price_update['date']) +
                      " send buy order for 10 shares price=%.2f" % (price_update['price']))
                self.position += 10
                self.cash -= cash_needed
            else:
                print('buy impossible because not enough cash')


        if action == 'sell':
            position_allowed=10
            if self.position-position_allowed>=-position_allowed:
                print(str(price_update['date'])+
                      " send sell order for 10 shares price=%.2f" % (price_update['price']))
                self.position -= position_allowed
                self.cash -= -position_allowed * price_update['price']
            else:
                print('sell impossible because not enough position')

        self.holdings = self.position * price_update['price']
        self.total = (self.holdings + self.cash)
        # print('%s total=%d, holding=%d, cash=%d' %
        #       (str(price_update['date']),self.total, self.holdings, self.cash))

        self.list_position.append(self.position)
        self.list_cash.append(self.cash)
        self.list_holdings.append(self.holdings)
        self.list_total.append(self.holdings+self.cash)


naive_backtester = None
nb_of_rows = 0

def test1():
    global naive_backtester
    global nb_of_rows
    nb_of_rows=10
    naive_backtester = ForLoopBackTester(strategy1())

def test2():
    global naive_backtester
    global nb_of_rows
    nb_of_rows=50
    naive_backtester = ForLoopBackTester(strategy1())

def test3():
    global naive_backtester
    global nb_of_rows
    nb_of_rows=10
    naive_backtester = ForLoopBackTester(strategy2())

def test4():
    global naive_backtester
    global nb_of_rows
    nb_of_rows=150
    naive_backtester = ForLoopBackTester(strategy2())

def test5():
    global naive_backtester
    global nb_of_rows
    nb_of_rows=600
    naive_backtester = ForLoopBackTester(strategy3())



if __name__ == '__main__':


    func_name = sys.stdin.readline().strip()
    test_func = globals()[func_name]
    test_func()
    market_data_header = input().strip()
    for _ in range(nb_of_rows):
        row = input().strip().split(',')

        date=row[0]
        high=row[1]
        low = row[2]
        closep=row[4]
        openp = row[3]
        volume = row[5]
        price=row[6]

        price_information={'date' : date,
                          'price' : float(price),
                           'high' : float(high),
                           'low': float(low),
                           'close' : float(closep),
                           'open' : float(openp),
                           'volume' : float(volume)}
        action = naive_backtester.onMarketDataReceived(price_information)
        naive_backtester.buy_sell_or_hold_something(price_information,action)



    print("PNL:%.2f" % (naive_backtester.list_total[-1] - 10000))  
        
        
